### This code requires the dictionaries extracted with the 'loading_in_tar' code. From these dictionaries, we extract the neural data for a given brain region. A filter is set to only take spikes that were given a phy annotation of 2, indicating a good sort. 

In [ ]:
## getting just VISp data (V1)
# end result: dictionary with keys: session and values: spike indices for the spikes that were in ROI

import numpy

vis_ctx = {}
sessions = np.arange(1, 41)

for s in sessions:
    print('session:', s)
    # find which channels are in V1
    rows = np.shape(channel_brainLocations[s])[0] # how many channels in this session
    channels = [] 
    for r in range(rows):
        if channel_brainLocations[s][r]=='VISp': # this string can be modified to find data for any region
            channels.append(r)
        else:
            continue
    if len(channels)<1:
        print('this session had no recordings in ROI')
        continue
    channels = np.asarray(channels)
    
    # now find all clusters who's peak value was at this channel
    clusters = []
    for chan in channels:
        more_rows = np.shape(clusters_peakChannel[s])[0] # how many clusters in this session
        for r2 in range(more_rows):
            if int(np.squeeze(clusters_peakChannel[s][r2])) == chan and int(np.squeeze(clusters_phy_annotation[s][r2])) == 2:
                clusters.append(r2)
            else:
                continue
        else:
            continue
    #print(clusters)
    clusters = np.asarray(clusters)
    
    # now find all spikes who were included in this cluster
    spikes = []
    for clus in clusters:
        print('cluster', clus, 'of', len(clusters))
        most_rows = np.shape(spikes_clusters[s])[0] # how many spikes in this session
        for r3 in range(most_rows):
            if int(np.squeeze(spikes_clusters[s][r3])) == clus:
                spikes.append(r3)
                
    spikes = np.asarray(spikes)
    print(spikes)
    vis_ctx[s]=spikes

### If you want to save the resulting dictionary to an .npy file, here's the code for that.

In [ ]:
# EXPORT V1 SPIKE DICTIONARY to file

os.chdir('/your/filepath/here')
import pickle

file_to_write = open("V1_spikes.npy", "wb")
pickle.dump(vis_ctx, file_to_write)

### Now, we need to make a new dictionary for the spike timestamps that only has the values for spikes in V1.

In [ ]:
# initialize dictionary with the same keys as vis_ctx
vis_timestamps = {}
keyList = vis_ctx.keys()
for i in keyList: 
    vis_timestamps[i] = None
    
# iterate over all the items in dictionary and filter items which have even keys
for (key, value) in vis_ctx.items(): 
    for (keyTS, valueTS) in vis_timestamps.items():
        if key == keyTS: # if keys are equal, move spiketime values to new dict
            spiketimes = []
            for v in value:
                spiketime = spikes_times[key][v]
                spiketimes.append(spiketime)
            vis_timestamps[key] = np.asarray(spiketimes)

print('done!')